In [1]:
import numpy as np
import cmath
import warnings
try:
    import scipy
except: 
    !pip install scipy
    
from scipy.integrate import quad
try:
    import sympy
except: 
    !pip install sympy 
from sympy import *

In [2]:
def complex_quadrature(func, lower, upper, **kwargs):
    '''Funkcja pozwalająca na obliczanie całki zespolonej'''
    def real_func(x):
        return np.real(func(x))
    def imag_func(x):
        return np.imag(func(x))
    real_integral = quad(real_func, lower, upper, **kwargs)
    imag_integral = quad(imag_func, lower, upper, **kwargs)
    return real_integral[0] + 1j*imag_integral[0]

In [3]:
class Wielomian:
    def __init__(self):
        self.lista_pierwiastkow = {} # słownik
        self.wspolczynniki = []
        self.stopien = None
        
    def load_polynomial(self, plik):
        with open(plik, 'r') as f:
            lines = f.read()  
        arr = [float(l) for l in lines.splitlines()]
        self.wspolczynniki = arr[1:]
        self.stopien = int(arr[0])
    
    def compute_f(self, z = Symbol('z')):
        if len(self.wspolczynniki) < self.stopien + 1:
            print(f'Podano za mało współczynników, potrzeba ich {self.stopien+1}. Popraw w pliku.')
            return "stop"

        if len(self.wspolczynniki) > self.stopien + 1:
            print(f'Podano więcej współczynników niż oczekiwana liczba: {self.stopien+1}. Popraw w pliku.')
            return "stop"
        
        f = np.sum([self.wspolczynniki[i] * np.power(z, i) for i in range(self.stopien+1)])
        return f
    
    def compute_df(self, fun, z = Symbol('z')):
        fp = fun.diff(z) 
        return fp
    
    def residuum(self, R, eps=1e-6):
        '''Metoda służąca do określenia liczby pierwiastków wielomianu p(x) znajdujących się w obszarze 
        ograniczonym okręgiem o środku w (0,0) i promieniu R.'''

        def p(x):
            return np.sum([self.wspolczynniki[i]*np.power(x,i) for i in range(self.stopien+1)])

        def p_prime(x):
            return np.sum([i*self.wspolczynniki[i]*np.power(x,i-1) for i in range(1, self.stopien+1)])

        def P(x):
            try:
                return p_prime(x)/p(x)
            except:
                return 0

        def gamma(x):
            '''Funkcja służąca do parametryzacji okręgu o środku w (0,0) i promieniu R'''
            return R*np.exp(1j*x) # R*e^{it}

        def gamma_prime(x):
            '''Pochodna z funkcji parametryzującej okrąg'''
            return R*1j*np.exp(1j*x) # R*i*e^{it}
        
        with warnings.catch_warnings():
            warnings.filterwarnings('error')

            try: 
                result = np.real(complex_quadrature(lambda x: P(gamma(x))*gamma_prime(x)/(2*np.pi*1j), 0, 2*np.pi))
                #warnings.warn(Warning())
                
                print("W obszarze ograniczonym okręgiem o środku w (0,0) i promieniu {:.4f} znajdziesz {} z {} pierwiastków wielomianu.".format(R, np.round(result,1), self.stopien))
            except Warning:
                print("Residuum: Jeden z pierwiastków wielomianu znajduje się na konturze obszaru (lub bardzo blisko), całka jest rozbieżna.")

    
    def Newton(self, fun, n_steps=5e3, eps=1e-10, decimal = 10):
        
        counter = 0
        k = 1
        exit = False
        startowy = False

        while(startowy==False):
            try:
                z0 = complex(input("Wprowadź punkt startowy (rzeczywisty lub zespolony): "))
                startowy = True
            except:
                print("Punkt startowy musi być typu 'complex'. Spróbuj ponownie:")
                continue
                
        z1 = z0 - 1 # inicjujemy dowolny punkt spełniający np.abs(z0 - z1) > eps
        f = fun
        df = self.compute_df(fun)
        z = Symbol('z')
        
        df = lambdify(z, df)
        f = lambdify(z, f) 
        
        # obserwujemy zbieżność metody poprzez warunek: (np.abs(z0 - z1) > eps)

        while( (k <= n_steps) and (np.abs(z0 - z1) > eps) ):
              
            try:
                z1 = z0 # zapamiętujemy ostatni punkt
                z0 = z0 - f(z0)/df(z0) # krok metody
                # print(z0)
            except:
                print("Pochodna w punkcie startowym wynosi 0!")
                exit = True
                break

            if np.abs(df(z0)) < eps:
                print("Zły punkt startowy, pochodna w kolejnym kroku jest zbyt bliska 0.")
                exit = True
                break
            
            k = k+1

        if exit:
            return 0
        
        '''Sprawdzamy, czy został przekroczony limit kroków'''
        if k > n_steps:
            print("Przekroczono limit kroków algorytmu, zły punkt startowy.")
            cont = input("Czy chcesz spróbować ponownie? [T/N] ")
            if cont.lower() in ["n", "no", "nie"]: 
                if len(self.lista_pierwiastkow) == 0:
                    print("Nie znaleziono żadnego pierwiastka tego wielomianu.")
                    return 0
                else:
                    print("Znalezione pierwiastki tego wielomianu to: ")
                    for pierw, krotnosc in self.lista_pierwiastkow.items():
                        print(f"z = {pierw}, krotność: {krotnosc}")
                    return 0

            else:
                self.Newton( fun )
        else:
            '''Jeśli limit kroków nie został przekroczony, to znaleziony pierwiastek dodajemy do listy 
            (o ile jeszcze go na niej nie ma)'''
            
            if len(self.lista_pierwiastkow) != 0:
                for z in self.lista_pierwiastkow.keys():
                    if np.abs(z0 - z) < eps*5e5:
                        print("Znaleziono pierwiastek wielokrotny.")
                        self.lista_pierwiastkow[z] += 1
                        break

                    else: 
                        print("Znaleziono nowy pierwiastek.")
                        self.lista_pierwiastkow[np.round(z0, decimal)] = 1
                        break

            else: 
                print("Znaleziono pierwszy pierwiastek.")
                self.lista_pierwiastkow[np.round(z0, decimal)] = 1
                

            '''Jeśli znaleźliśmy n pierwiastków wielomianu stopnia n, to kończymy'''
            if sum(self.lista_pierwiastkow.values()) == self.stopien:
                print("Pierwiastki tego wielomianu to: ")
                for pierw, krotnosc in self.lista_pierwiastkow.items():
                    print(f"z = {pierw}, krotność: {krotnosc}")
                return 1
            
            else:
                self.Newton( fun/(z-z0) ) # fun wciąż działa na symbolach
                
# Tworzymy obiekt klasy Wielomian, konstruktor nie wymaga podania argumentów            
polynomial = Wielomian()

# Z pliku wielomian.txt ładujemy współczynniki do naszego obiektu polynomial
polynomial.load_polynomial('wielomian.txt')

# Używamy metody compute_f(), aby symbolicznie zapisać wzór wielomianu
f = polynomial.compute_f() 

# Jeśli polynomial.compute_f() zwróci nam "stop", to znaczy, że popełniliśmy błąd w pliku 'wielomian.txt.'
if f != "stop":
    # Sprawdźmy, z jakim punktem startowym warto zacząć szukać pierwiastków wielomianu f
    polynomial.residuum(R = 7)

    message = input("Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: ")
    if message.lower() in ["t", "tak", "yes"]:
        polynomial.Newton(f)

W obszarze ograniczonym okręgiem o środku w (0,0) i promieniu 7.0000 znajdziesz 2.0 z 2 pierwiastków wielomianu.
Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: T
Wprowadź punkt startowy (rzeczywisty lub zespolony): 2j
Znaleziono pierwszy pierwiastek.
Wprowadź punkt startowy (rzeczywisty lub zespolony): 6
Znaleziono nowy pierwiastek.
Pierwiastki tego wielomianu to: 
z = (-3+0j), krotność: 1
z = (-4-0j), krotność: 1


## Dodatkowe przykłady
#### Żeby zadziałały, musi zostać przed nimi wywołana pierwsza komórka kodu

In [4]:
polynomial_2 = Wielomian()
polynomial_2.load_polynomial('wielomian2.txt')
f = polynomial_2.compute_f() 

if f != "stop":
    polynomial_2.residuum(R = 10)
    message = input("Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: ")
    if message.lower() in ["t", "tak", "yes"]:
        polynomial_2.Newton(f)

W obszarze ograniczonym okręgiem o środku w (0,0) i promieniu 10.0000 znajdziesz 3.0 z 3 pierwiastków wielomianu.
Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: T
Wprowadź punkt startowy (rzeczywisty lub zespolony): 3
Znaleziono pierwszy pierwiastek.
Wprowadź punkt startowy (rzeczywisty lub zespolony): 4j
Znaleziono pierwiastek wielokrotny.
Wprowadź punkt startowy (rzeczywisty lub zespolony): 5-j
Znaleziono pierwiastek wielokrotny.
Pierwiastki tego wielomianu to: 
z = (-0.9999934481+0j), krotność: 3


In [5]:
# Tworzymy obiekt klasy Wielomian, konstruktor nie wymaga podania argumentów            
polynomial_3 = Wielomian()

# Z pliku wielomian.txt ładujemy współczynniki do naszego obiektu polynomial
polynomial_3.load_polynomial('wielomian3.txt')

# Używamy metody compute_f(), aby symbolicznie zapisać wzór wielomianu
f = polynomial_3.compute_f() 

# Jeśli polynomial.compute_f() zwróci nam "stop", to znaczy, że popełniliśmy błąd w pliku 'wielomian.txt.'
if f != "stop":
    # Sprawdźmy, z jakim punktem startowym warto zacząć szukać pierwiastków wielomianu f
    polynomial_3.residuum(R = 1)
    polynomial_3.residuum(R = 2.1)

    message = input("Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: ")
    if message.lower() in ["t", "tak", "yes"]:
        polynomial_3.Newton(f)

W obszarze ograniczonym okręgiem o środku w (0,0) i promieniu 1.0000 znajdziesz 0.0 z 2 pierwiastków wielomianu.
W obszarze ograniczonym okręgiem o środku w (0,0) i promieniu 2.1000 znajdziesz 2.0 z 2 pierwiastków wielomianu.
Czy chcesz rozpocząć szukanie pierwiastków? [T/N]: T
Wprowadź punkt startowy (rzeczywisty lub zespolony): 2.1j
Znaleziono pierwszy pierwiastek.
Wprowadź punkt startowy (rzeczywisty lub zespolony): 0.8+j
Znaleziono nowy pierwiastek.
Pierwiastki tego wielomianu to: 
z = 2j, krotność: 1
z = -2j, krotność: 1
